In [1]:
#check librosa stft and istft
import matplotlib
matplotlib.use('Agg')
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np

from dataio import readdata, readdata2, readlabels, writedata
from datafilters import apply_dc_filter, apply_dwt_filter, apply_stfft_filter

In [2]:
#Hyperparams

#DC Filter
enable_dc = True
dc_lowcut = 3.0
dc_highcut = 30.0
dc_order = 2
dc_type = "bandpass"
dc_func_type = "butter"

#DWT Filter
enable_dwt = True
dwt_type = "db2"
dwt_level = 4
dwt_thresh_func = "soft"
dwt_thresh_type = "rigrsure"

#STFT Filter
enable_fft = True
fft_window = 0.50
fft_step = 0.25
fft_thresh = 8.0
fft_set_thresh = 0.0

In [3]:
#Plots for comparison
#plt.figure(1);
#plt.figure(figsize=(10,12));

#Raw EEG Signal
#plt.subplot(411);
#plt.plot(y1);
#plt.title("Raw EEG")

#DC Filter
#plt.subplot(412);
#plt.plot(y2);
#plt.title("EEG + DC Filter");

#Discrete Wavelet Transform
#plt.subplot(413);
#plt.plot(y3);
#plt.title("EEG + DC + Discrete Wavelet Transform");

#Fast Fourier Transform
#plt.subplot(414);
#plt.plot(y4);
#plt.title("EEG + DC + DWT + Fast Fourier Transform");

#plt.show()

In [4]:
dataset = readdata("sampledata")
#Constants
fs = 250.0 #Frequency in Hz
sample_time = dataset.shape[1]/fs #Total time for sample

#To get alpha and beta waves
fft_thresh = 2
enable_fft = True
dataset.flags['WRITEABLE'] = True
for i in range(0,dataset.shape[0]):
    for j in range(0,dataset.shape[2]):
        if enable_dc:
            dataset[i,:,j] = apply_dc_filter(dataset[i,:,j], fs, dc_lowcut, dc_highcut, dc_order, dc_type, dc_func_type)
        if enable_dwt:
            dataset[i,:,j] = apply_dwt_filter(dataset[i,:,j], dwt_type, dwt_level, dwt_thresh_func, dwt_thresh_type)
        if enable_fft:
            dataset[i,:,j] = apply_stfft_filter(dataset[i,:,j], fs, sample_time, fft_window, fft_step, fft_thresh, fft_set_thresh)
        #Normalize
        dataset[i,:,j] = dataset[i,:,j]/np.linalg.norm(dataset[i,:])

In [5]:
test_size = 0.20
valid_size = 0.20
labels = readlabels("sampleinput")

# Test Set
dataindex = range(0,dataset.shape[0])
test_index = np.random.choice(dataindex, int(dataset.shape[0]*test_size))
test_dataset = dataset[test_index,:,:]
test_labels = labels[test_index]
dataset = np.delete(dataset, test_index, axis = 0)
labels = np.delete(labels, test_index, axis = 0)

# Validation Set
dataindex = range(0,dataset.shape[0])
valid_index = np.random.choice(dataindex, int(dataset.shape[0]*valid_size))
valid_dataset = dataset[valid_index,:,:]
valid_labels = labels[valid_index]

# Train Set
train_dataset = dataset
train_labels = labels

print('Training:', train_dataset.shape, train_labels.shape)
print('Validation:', valid_dataset.shape, valid_labels.shape)
print('Testing:', test_dataset.shape, test_labels.shape)

Training: (105, 1130, 8) (105,)
Validation: (21, 1130, 8) (21,)
Testing: (26, 1130, 8) (26,)


In [6]:
#Write training
writedata("./curated/train_dataset", train_dataset)
writedata("./curated/train_labels", train_labels)
#Write validation
writedata("./curated/valid_dataset", valid_dataset)
writedata("./curated/valid_labels", valid_labels)
#Write test
writedata("./curated/test_dataset", test_dataset)
writedata("./curated/test_labels", test_labels)
#plt.figure(figsize=(15,8))
#plt.plot(dataset[0,:,0])
#plt.show()

In [2]:
#Read training
train_dataset = readdata2("./curated/train_dataset")
train_labels = readlabels("./curated/train_labels")
#Read validation
valid_dataset = readdata2("./curated/valid_dataset")
valid_labels = readlabels("./curated/valid_labels")
#Read test
test_dataset = readdata2("./curated/test_dataset")
test_labels = readlabels("./curated/test_labels")

print('Training:', train_dataset.shape, train_labels.shape)
print('Validation:', valid_dataset.shape, valid_labels.shape)
print('Testing:', test_dataset.shape, test_labels.shape)

Training: (105, 1130, 8) (105,)
Validation: (21, 1130, 8) (21,)
Testing: (26, 1130, 8) (26,)
